Copyright (c) 2020, NVIDIA CORPORATION.
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [440]:
import matplotlib.pyplot as plt
%matplotlib inline  

import pandas as pd
import numpy as np
import gc

In [441]:
#Reload the output of previous notebook

df = pd.read_pickle('tmp_small.pkl')
print( df.shape )
df.head(50)

(10000, 24)


,hashtags,tweet_id,media,links,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,...,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id
0,A3D61C03DBCE2F920350379414E0048B\t867E9AB76EFF...,0,5,NaN,NaN,2,11,1581472519,0,11005,...,4,6,False,1568940527,True,0,0,0,0,0
1,NaN,1,0,NaN,NaN,0,59,1581124265,1,425,...,225,185,False,1541013180,True,0,0,0,1581174423,1
2,NaN,2,0,NaN,NaN,2,38,1581166895,2,298336,...,4,72,False,1573996260,False,0,0,0,1581201675,2
3,NaN,3,0,NaN,NaN,2,11,1581375781,3,1115,...,780,440,False,1432084055,True,0,0,0,1581437254,3
4,NaN,4,0,NaN,NaN,1,54,1580996019,4,63473,...,180,32,False,1490688761,False,0,0,0,0,4
5,NaN,5,7,NaN,NaN,2,59,1581004518,5,29358,...,929,928,False,1577292324,True,1581008849,0,0,0,5
6,D8788CF06B4D91C8A6CED6296546409D\t49A1E0B17ED6...,6,0,NaN,NaN,1,54,1581209689,6,562,...,929,928,False,1577292324,True,0,0,0,1581209817,6
7,NaN,7,0,NaN,NaN,2,54,1581511341,7,529009,...,139,952,False,1335110299,False,0,0,0,1581515122,7
8,38F9B8B232E7F776F62141A30ED2561A,8,9,NaN,NaN,2,59,1581193484,8,26730,...,286,522,False,1439811227,False,0,0,0,0,8
9,NaN,9,0,NaN,NaN,2,47,1581433245,9,2664,...,48,140,False,1568691506,False,0,0,0,0,9


In [442]:
%%time

df['len_hashtags'] = df['hashtags'].apply(lambda x: str(x).count('\t')+1 if not(pd.isnull(x)) else 0)
df['len_domains']  = df['domains'].apply(lambda x: str(x).count('\t')+1 if not(pd.isnull(x)) else 0)
df['len_links']    = df['links'].apply(lambda x: str(x).count('\t')+1 if not(pd.isnull(x)) else 0)

Wall time: 40 ms


In [443]:
%%time
var = df['domains'].fillna('').values.copy()
gc.collect()

PD = {}
null = ''
PD[null] = [0,0]
count = 1
for vs in var:
    if vs != null:
        for v in vs.split('\t'):
            if v not in PD:
                PD[v] = [count,1]
                count +=1
            else:
                x = PD[v]
                x[1] += 1
                PD[v] = x
    else:
        x = PD[null]
        x[1] += 1
        PD[null] = x
        
len(PD)

Wall time: 134 ms


840

In [444]:
list(PD.items())[:10]

[('', [0, 8388]),
 ('215887F9F98736B07C823045CA9FA2EB', [1, 1]),
 ('3E17824EEB6725B91628B9C13C28894B', [2, 1]),
 ('FECA6F2E8244F2294BD2CE957C0602A9', [3, 65]),
 ('79F1DCA7DD157C768CCDF4817A0D3999', [4, 16]),
 ('96A073F6BDEEAEDF58E1C9DD1F41F4F8', [5, 1]),
 ('D0A12E9B144295D771A6A6D058337F65', [6, 1]),
 ('1336DD7055851C3EF96BBF8A6B0447DE', [7, 2]),
 ('8A67C9075AF148F8C410E56DC9E6BA2C', [8, 2]),
 ('E91CDEC8DC7ABF30592FA024616FF970', [9, 108])]

In [445]:
%%time
vari = []
for vs in var:
    if vs != null:
        li=[]
        lf=[]
        for v in vs.split('\t'):
            if v!='':
                li.append(PD[v][0])
                lf.append(-PD[v][1])
        vari.append( list(np.array(li)[np.argsort(lf)].astype(np.int32) ) )
    else:
        vari.append( [0] )
del PD
gc.collect()

len(vari), vari[:10]

Wall time: 161 ms


(10000, [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]])

In [446]:
#Get only the most frequent domain
df['domains'] = np.array( [v[0] for v in vari ] ).astype( np.int32 )
gc.collect()
del vari, var
gc.collect()

0

In [447]:
%%time
var = df['links'].fillna('').values.copy()
gc.collect()

PD = {}
null = ''
PD[null] = [0,0]
count = 1
for vs in var:
    if vs != null:
        for v in vs.split('\t'):
            if v not in PD:
                PD[v] = [count,1]
                count +=1
            else:
                x = PD[v]
                x[1] += 1
                PD[v] = x
    else:
        x = PD[null]
        x[1] += 1
        PD[null] = x
        
len(PD),list(PD.items())[:10]

Wall time: 133 ms


(1636,
 [('', [0, 8388]),
  ('172DBB18AD23E0AA09074F0C5DBA23D6', [1, 1]),
  ('F4A3AA58F9231C4B9FDE4FCDA1BA557A', [2, 1]),
  ('3A8007779EA9AFDC7196FC405506A61B', [3, 1]),
  ('CBFC53143ED4B60C86C6716CC252687F', [4, 1]),
  ('8D976CF2A1ACFB58A10C4C8E0EE71C9C', [5, 1]),
  ('41DD3CDA748D3F95900DF8E69965DE6C', [6, 1]),
  ('EAED1FB7E20CDE04095A516621E03C24', [7, 1]),
  ('F3D35089AD651FAA1F8FBB020AF10ED7', [8, 1]),
  ('D7BA8D1BE90C19C024DE060DEE6D0E95', [9, 1])])

In [448]:
%%time
vari = []
for vs in var:
    if vs != null:
        li=[]
        lf=[]
        for v in vs.split('\t'):
            if v!='':
                li.append(PD[v][0])
                lf.append(-PD[v][1])
        vari.append( list(np.array(li)[np.argsort(lf)].astype(np.int32) ) )
    else:
        vari.append( [0] )
del PD
gc.collect()

len(vari), vari[:10]

Wall time: 151 ms


(10000, [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]])

In [449]:
#Get only the most frequent link
df['links'] = np.array( [v[0] for v in vari ] ).astype( np.int32 )
gc.collect()
del vari, var
gc.collect()

0

In [450]:
%%time
var = df['hashtags'].fillna('').values.copy()
gc.collect()

PD = {}
null = ''
PD[null] = [0,0]
count = 1
for vs in var:
    print(len(vs))
    if vs != null:
        for v in vs.split('\t'):
            if v not in PD:
                PD[v] = [count,1]
                count +=1
            else:
                x = PD[v]
                x[1] += 1
                PD[v] = x
            print(v)
    else:
        x = PD[null]
        x[1] += 1
        PD[null] = x
        
len(PD),list(PD.items())[:10]

5C
654820081BE64C90D8F0BA2C077E6AF7
0
0
0
0
98
1C6502F1D678D865E348F3D18C79DB9C
BA53A319D2EB405B00C9FB7EFC3F2360
4FB84C8A3FB8A11A0AE95676C358D9F5
0
0
230
D53B42C55280883325CAB2F23A886F54
9B4803E0812113E9C564EA274955170E
6EA498ED434968DD7A4F562ECECF250B
5DE2CC196CDF7727AA02DAABB3122EB2
3883B38570FE118273195CBF1956E4D6
536F81DD713D82347729451ED7EF3E1C
E9F208B5771DD0975051C6810838A97F
0
32
E9888019A3F0E88F4DA42FA516DE9BB5
0
0
0
32
494993F1099F825A7D96A763172E1116
0
0
0
32
F3A1A12C43635E763556CBE77B729C86
0
0
0
0
0
0
0
0
65
05D6A1A9E943A5CA338D79C911761D9A
393790595D3452C87811C225D546F35C
0
197
2C7EE65252C3116E27175386B4920206
4CDD639E0F497FD5FBB6A1F9474224F6
D39BEEEE4753413558E59C74F1645C8F
46BE3A3A9E0FDEBC4BDD28606B218113
4AA36F2C1CCC37F5C4048A1D0A1A2E66
EA9512A1ED9E333A4E1372B22F721C77
0
0
0
0
0
0
0
0
0
0
65
4BB6DBA6E4B411E4D9658A3E21665578
F849F4F0D97B56AB2E537DC65C2FEDCA
0
65
04402C9D3E548D2C46D91798EDF32047
3B5EE35C78DE85E509A2DEACAF01A299
0
0
0
32
15647F4D4E42D184C204A85486438935
32

(3567,
 [('', [0, 7839]),
  ('A3D61C03DBCE2F920350379414E0048B', [1, 1]),
  ('867E9AB76EFF8C7EF9DE926710DDBC99', [2, 1]),
  ('D8788CF06B4D91C8A6CED6296546409D', [3, 2]),
  ('49A1E0B17ED69FDB02A736B85F953BFD', [4, 1]),
  ('38F9B8B232E7F776F62141A30ED2561A', [5, 1]),
  ('CD46B9E19F28918E38A893A71D7323ED', [6, 1]),
  ('0C2CE69D7151A003C99145F2071FA76E', [7, 1]),
  ('053074B6C39ADD256B984F5498E431EC', [8, 3]),
  ('46A5CAAF8FE1D26E72A54A96285C8CAC', [9, 3])])

In [451]:
%%time
vari = []
for vs in var:
    if vs != null:
        li=[]
        lf=[]
        for v in vs.split('\t'):
            if v!='':
                li.append(PD[v][0])
                lf.append(-PD[v][1])
        vari.append( list(np.array(li)[np.argsort(lf)].astype(np.int32) ) )
    else:
        vari.append( [0] )
del PD
gc.collect()

len(vari), vari[:10]

Wall time: 183 ms


(10000, [[1, 2], [0], [0], [0], [0], [0], [3, 4], [0], [5], [0]])

In [452]:
#Get only the most frequent hashtag
df['hashtags'] = np.array( [v[0] for v in vari ] ).astype( np.int32 )
gc.collect()
del vari, var
gc.collect()

0

In [453]:
df.tail(40)

,hashtags,tweet_id,media,links,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,...,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id,len_hashtags,len_domains,len_links
9960,0,9914,0,1623,111,2,54,1581437130,8983,583,...,1572960672,True,0,0,0,1581474903,9960,0,1,1
9961,0,9915,5,0,0,2,54,1581291119,8984,92352,...,1235358459,False,0,0,0,1581299011,9961,0,0,0
9962,0,9916,9,1624,4,2,54,1581358030,1100,26671207,...,1235358459,False,0,0,0,0,9962,0,1,1
9963,0,9917,7,0,0,2,11,1581067247,8985,1045,...,1421898529,True,0,0,0,1581068337,9963,0,0,0
9964,0,9918,0,0,0,0,54,1581247615,8986,1403,...,1477895824,True,0,0,0,0,9964,0,0,0
9965,3547,9919,5,0,0,2,54,1581016172,8987,9052,...,1426042348,False,0,0,0,1581040495,9965,6,0,0
9966,3553,9920,7,1625,832,2,3,1580987757,8988,2129,...,1251241503,True,0,0,0,0,9966,2,1,1
9967,0,9921,0,0,0,0,54,1581488191,8989,50454,...,1243742630,False,0,0,0,0,9967,0,0,0
9968,0,9922,0,0,0,1,54,1581438595,8990,165,...,1546055327,True,0,0,0,0,9968,0,0,0
9969,0,9923,7,0,0,2,4,1581165574,8991,12478,...,1543909020,True,0,0,0,1581174625,9969,0,0,0


In [454]:
df.dtypes

hashtags               int32
tweet_id              uint32
media                  uint8
links                  int32
domains                int32
tweet_type             uint8
language               uint8
timestamp             uint32
a_user_id             uint32
a_follower_count      uint32
a_following_count     uint32
a_is_verified           bool
a_account_creation    uint32
b_user_id             uint32
b_follower_count      uint32
b_following_count     uint32
b_is_verified           bool
b_account_creation    uint32
b_follows_a             bool
reply                 uint32
retweet               uint32
retweet_comment       uint32
like                  uint32
id                    uint32
len_hashtags           int64
len_domains            int64
len_links              int64
dtype: object

In [455]:
train = df.loc[(df.tr == 0)].copy()
test0 = df.loc[(df.tr == 1)].copy()
test1 = df.loc[(df.tr == 2)].copy()

train.shape, test0.shape, test1.shape

AttributeError: 'DataFrame' object has no attribute 'tr'

In [34]:
train.to_parquet( 'train.parquet' )
test0.to_parquet( 'valid.parquet' )
test1.to_parquet( 'test.parquet' )
gc.collect()

0